In [1]:
# strucutre of the FCSTNORM2021 FILES
import pickle

with open('/content/FCSTNorm2021.pkl', 'rb') as f:
  data = pickle.load(f)

In [2]:
data

{'Convective available potential energy': {'min': 0.0,
  'max': 5060.9165,
  'mean': 353.36827374192967,
  'std': 576.8885141782403},
 'Convective precipitation (water)': {'min': 0.0,
  'max': 10942.708,
  'mean': 221.19678806460806,
  'std': 510.6248432421585},
 'Medium cloud cover': {'min': 0.0,
  'max': 100.0,
  'mean': 18.048821338712706,
  'std': 26.28508984502161},
 'Surface pressure': {'min': 66547.43,
  'max': 102703.49,
  'mean': 94929.37199037352,
  'std': 5486.265313158451},
 'Upward short-wave radiation flux': {'min': 0.0,
  'max': 289.14667,
  'mean': 43.595493802276515,
  'std': 57.628844801216125},
 'Downward short-wave radiation flux': {'min': 0.35599998,
  'max': 722.91364,
  'mean': 264.80159996631585,
  'std': 256.85152430546674},
 '2 metre temperature': {'min': 274.87875,
  'max': 319.77393,
  'mean': 299.4504116180272,
  'std': 5.612859447433858},
 'Cloud water': {'min': 0.0,
  'max': 2.7919734,
  'mean': 0.04932853633297358,
  'std': 0.11816838266414283},
 'Precip

In [3]:
!pip install -q -U zarr xarray gcsfs
import xarray as xr
import gcsfs
import zarr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 64.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 21.0.0 which is incompatible.


In [27]:
# define the neuralgcm path data
ngcm_path = 'gs://neuralgcm/amip_runs/v1_precip_stochastic_2_8_deg/2001-to-2021_128x64_gauss_37-level_stride3h.zarr'
#load the data
# ngcm_data = xr.open_zarr(ngcm_path,chunks={"time":100})
ngcm_data = xr.open_zarr(ngcm_path)

<ipython-input-27-cd22bc7a61c8>:5: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ngcm_data = xr.open_zarr(ngcm_path)


In [28]:
#select Horn Africa data
ngcm_data_africa = ngcm_data.sel(latitude=slice(-20,30),longitude=slice(15,60))


In [29]:
#select the climate timeset where prediction_timedelta is a multiple of 6h

# Convert prediction_timedelta to a pandas Series for easier manipulation
prediction_timedelta_series = pd.Series(ngcm_data_africa.prediction_timedelta.values)

# Select values where prediction_timedelta is a multiple of 6 hours (in seconds)
selected_indices = prediction_timedelta_series[prediction_timedelta_series % pd.Timedelta(hours=6) == pd.Timedelta(0)].index

# Use the selected indices to filter the entire dataset
selected_data = ngcm_data_africa.isel(prediction_timedelta=selected_indices)

selected_data

<xarray.Dataset> Size: 29GB
Dimensions:                              (time: 37,
                                          prediction_timedelta: 29204,
                                          surface: 1, longitude: 16,
                                          latitude: 18, level: 3)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
  * level                                (level) int64 24B 500 700 850
  * longitude                            (longitude) float64 128B 16.88 ... 5...
  * prediction_timedelta                 (prediction_timedelta) timedelta64[ns] 234kB ...
  * surface                              (surface) int64 8B 1
  * time                                 (time) datetime64[ns] 296B 2001-01-0...
Data variables:
    evaporation                          (time, prediction_timedelta, surface, longitude, latitude) float32 1GB dask.array<chunksize=(1, 8, 1, 16, 18), meta=np.ndarray>
    geopotential                         (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (time, prediction_timedelta, surface, longitude, latitude) float32 1GB dask.array<chunksize=(1, 8, 1, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    specific_humidity                    (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    temperature                          (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (time, prediction_timedelta, level, longitude, latitude) float32 4GB dask.array<chunksize=(1, 8, 3, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

In [30]:
# Create the date dimension:
base_date = pd.Timestamp('2001-01-01')
dates = base_date + selected_data.prediction_timedelta.values
selected_data = selected_data.assign_coords(date=('prediction_timedelta', dates))
selected_data = selected_data.swap_dims({'prediction_timedelta': 'date'})
selected_data = selected_data.sel(date=slice('2018-01-01', '2018-12-31'),time=selected_data.time[-1],surface=selected_data.surface[0])
selected_data

<xarray.Dataset> Size: 39MB
Dimensions:                              (date: 1460, longitude: 16,
                                          latitude: 18, level: 3)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
  * level                                (level) int64 24B 500 700 850
  * longitude                            (longitude) float64 128B 16.88 ... 5...
    prediction_timedelta                 (date) timedelta64[ns] 12kB 6209 day...
    surface                              int64 8B 1
    time                                 datetime64[ns] 8B 2001-12-27
  * date                                 (date) datetime64[ns] 12kB 2018-01-0...
Data variables:
    evaporation                          (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    geopotential                         (date, level, longitude, latitude) float32 5MB dask.array<chunksize=(4, 3, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (date, level, longitude, latitude) float32 5MB dask.array<chunksize=(4, 3, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (date, level, longitude, latitude) float32 5MB dask.array<chunksize=(4, 3, 16, 18), meta=np.ndarray>
    specific_humidity                    (date, level, longitude, latitude) float32 5MB dask.array<chunksize=(4, 3, 16, 18), meta=np.ndarray>
    temperature                          (date, level, longitude, latitude) float32 5MB dask.array<chunksize=(4, 3, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (date, level, longitude, latitude) float32 5MB dask.array<chunksize=(4, 3, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (date, level, longitude, latitude) float32 5MB dask.array<chunksize=(4, 3, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

# selected our variable of interest

In [33]:
# Sélect one level and one surface(surface=1, level=0)
ds_subset_1 = selected_data.sel(level=selected_data.level[0])
ds_subset_1

<xarray.Dataset> Size: 15MB
Dimensions:                              (date: 1460, longitude: 16,
                                          latitude: 18)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
    level                                int64 8B 500
  * longitude                            (longitude) float64 128B 16.88 ... 5...
    prediction_timedelta                 (date) timedelta64[ns] 12kB 6209 day...
    surface                              int64 8B 1
    time                                 datetime64[ns] 8B 2001-12-27
  * date                                 (date) datetime64[ns] 12kB 2018-01-0...
Data variables:
    evaporation                          (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    geopotential                         (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_humidity                    (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    temperature                          (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

In [35]:
# Sélect one level and one surface(surface=1, level=0)
ds_subset_2 = selected_data.sel(level=selected_data.level[1])
ds_subset_2

<xarray.Dataset> Size: 15MB
Dimensions:                              (date: 1460, longitude: 16,
                                          latitude: 18)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
    level                                int64 8B 700
  * longitude                            (longitude) float64 128B 16.88 ... 5...
    prediction_timedelta                 (date) timedelta64[ns] 12kB 6209 day...
    surface                              int64 8B 1
    time                                 datetime64[ns] 8B 2001-12-27
  * date                                 (date) datetime64[ns] 12kB 2018-01-0...
Data variables:
    evaporation                          (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    geopotential                         (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_humidity                    (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    temperature                          (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

In [36]:
# Sélect one level and one surface(surface=1, level=0)
ds_subset_3 = selected_data.sel(level=selected_data.level[2])
ds_subset_3

<xarray.Dataset> Size: 15MB
Dimensions:                              (date: 1460, longitude: 16,
                                          latitude: 18)
Coordinates:
  * latitude                             (latitude) float64 144B -18.14 ... 29.3
    level                                int64 8B 850
  * longitude                            (longitude) float64 128B 16.88 ... 5...
    prediction_timedelta                 (date) timedelta64[ns] 12kB 6209 day...
    surface                              int64 8B 1
    time                                 datetime64[ns] 8B 2001-12-27
  * date                                 (date) datetime64[ns] 12kB 2018-01-0...
Data variables:
    evaporation                          (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    geopotential                         (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    precipitation_cumulative_mean        (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_cloud_ice_water_content     (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_cloud_liquid_water_content  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    specific_humidity                    (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    temperature                          (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    u_component_of_wind                  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
    v_component_of_wind                  (date, longitude, latitude) float32 2MB dask.array<chunksize=(4, 16, 18), meta=np.ndarray>
Attributes:
    experiment_id:  121052169
    worker_id:      2

In [37]:
#without pressure level
evaporation=selected_data.evaporation
precipitation_cumulative_mean=selected_data.precipitation_cumulative_mean

In [51]:
evaporation

<xarray.DataArray 'evaporation' (date: 1460, longitude: 16, latitude: 18)> Size: 2MB
dask.array<getitem, shape=(1460, 16, 18), dtype=float32, chunksize=(8, 16, 18), chunktype=numpy.ndarray>
Coordinates:
  * latitude              (latitude) float64 144B -18.14 -15.35 ... 26.51 29.3
  * longitude             (longitude) float64 128B 16.88 19.69 ... 56.25 59.06
    prediction_timedelta  (date) timedelta64[ns] 12kB 6209 days 00:00:00 ... ...
    surface               int64 8B 1
    time                  datetime64[ns] 8B 2001-12-27
  * date                  (date) datetime64[ns] 12kB 2018-01-01 ... 2018-12-3...

In [39]:
precipitation_cumulative_mean

<xarray.DataArray 'precipitation_cumulative_mean' (date: 1460, longitude: 16,
                                                   latitude: 18)> Size: 2MB
dask.array<getitem, shape=(1460, 16, 18), dtype=float32, chunksize=(8, 16, 18), chunktype=numpy.ndarray>
Coordinates:
  * latitude              (latitude) float64 144B -18.14 -15.35 ... 26.51 29.3
  * longitude             (longitude) float64 128B 16.88 19.69 ... 56.25 59.06
    prediction_timedelta  (date) timedelta64[ns] 12kB 6209 days 00:00:00 ... ...
    surface               int64 8B 1
    time                  datetime64[ns] 8B 2001-12-27
  * date                  (date) datetime64[ns] 12kB 2018-01-01 ... 2018-12-3...

In [40]:
#subset at pressure 500
specific_cloud_ice_water_content_500=ds_subset_1.specific_cloud_ice_water_content
u_component_of_wind_500=ds_subset_1.u_component_of_wind
v_component_of_wind_500=ds_subset_1.v_component_of_wind

In [41]:
#subset at pressure 700
specific_cloud_ice_water_content_700=ds_subset_2.specific_cloud_ice_water_content
u_component_of_wind_700=ds_subset_2.u_component_of_wind
v_component_of_wind_700=ds_subset_2.v_component_of_wind

In [42]:
#subset at pressure 850
specific_cloud_ice_water_content_850=ds_subset_3.specific_cloud_ice_water_content
u_component_of_wind_850=ds_subset_3.u_component_of_wind
v_component_of_wind_850=ds_subset_3.v_component_of_wind

In [44]:
from tqdm import tqdm

variables = {
    'evaporation': evaporation,
    'precipitation_cumulative_mean': precipitation_cumulative_mean,
    'specific_cloud_ice_water_content_500': specific_cloud_ice_water_content_500,
    'u_component_of_wind_500': u_component_of_wind_500,
    'v_component_of_wind_500': v_component_of_wind_500,
    'specific_cloud_ice_water_content_700': specific_cloud_ice_water_content_700,
    'u_component_of_wind_700': u_component_of_wind_700,
    'v_component_of_wind_700': v_component_of_wind_700,
    'specific_cloud_ice_water_content_850': specific_cloud_ice_water_content_850,
    'u_component_of_wind_850': u_component_of_wind_850,
    'v_component_of_wind_850': v_component_of_wind_850,
}
output_dict = {}
for var_name, var_data in tqdm(variables.items(), desc="Calculating statistics"):
  output_dict[var_name] = {
      'min': float(var_data.min().values),
      'max': float(var_data.max().values),
      'mean': float(var_data.mean().values),
      'std': float(var_data.std().values),
  }

output_dict

Calculating statistics: 100%|██████████| 11/11 [04:27<00:00, 24.28s/it]


{'evaporation': {'min': -0.0002590485382825136,
  'max': 6.542875053128228e-05,
  'mean': -2.763666634564288e-05,
  'std': 3.2645803003106266e-05},
 'precipitation_cumulative_mean': {'min': 0.041313234716653824,
  'max': 46.9560661315918,
  'mean': 14.184422492980957,
  'std': 11.348319053649902},
 'specific_cloud_ice_water_content_500': {'min': -8.934714969655033e-06,
  'max': 7.734131213510409e-05,
  'mean': 9.304903869633563e-07,
  'std': 3.1453987503482495e-06},
 'u_component_of_wind_500': {'min': -25.939992904663086,
  'max': 49.0439567565918,
  'mean': 2.1932971477508545,
  'std': 8.303388595581055},
 'v_component_of_wind_500': {'min': -33.8306999206543,
  'max': 33.81747817993164,
  'mean': -0.3208630383014679,
  'std': 4.3230061531066895},
 'specific_cloud_ice_water_content_700': {'min': -7.905081474746112e-06,
  'max': 2.1997100702719763e-05,
  'mean': -1.7136811436557764e-07,
  'std': 1.1957555443586898e-06},
 'u_component_of_wind_700': {'min': -23.81795883178711,
  'max': 31

In [45]:

with open('/content/neuralgcm_Horn_Africa_2018_stats.pkl', 'wb') as f:
  pickle.dump(output_dict, f)

In [50]:
# prompt: import the file
import pickle

with open('neuralgcm_Horn_Africa_2018_stats.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
loaded_data

{'evaporation': {'min': -0.0002590485382825136,
  'max': 6.542875053128228e-05,
  'mean': -2.763666634564288e-05,
  'std': 3.2645803003106266e-05},
 'precipitation_cumulative_mean': {'min': 0.041313234716653824,
  'max': 46.9560661315918,
  'mean': 14.184422492980957,
  'std': 11.348319053649902},
 'specific_cloud_ice_water_content_500': {'min': -8.934714969655033e-06,
  'max': 7.734131213510409e-05,
  'mean': 9.304903869633563e-07,
  'std': 3.1453987503482495e-06},
 'u_component_of_wind_500': {'min': -25.939992904663086,
  'max': 49.0439567565918,
  'mean': 2.1932971477508545,
  'std': 8.303388595581055},
 'v_component_of_wind_500': {'min': -33.8306999206543,
  'max': 33.81747817993164,
  'mean': -0.3208630383014679,
  'std': 4.3230061531066895},
 'specific_cloud_ice_water_content_700': {'min': -7.905081474746112e-06,
  'max': 2.1997100702719763e-05,
  'mean': -1.7136811436557764e-07,
  'std': 1.1957555443586898e-06},
 'u_component_of_wind_700': {'min': -23.81795883178711,
  'max': 31

In [57]:
# prompt: melvin_aims_bucket/evaporation/2001/ngcm_evaporation_2.8deg_6h_GHA_20010101_00h.nc charger et affciher le fichier suivant a partir du bucket

!gsutil cp gs://melvin_aims_bucket/evaporation/2001/ngcm_evaporation_2.8deg_6h_GHA_20010101_06h.nc /tmp/

import xarray as xr

ds = xr.open_dataset('/tmp/ngcm_evaporation_2.8deg_6h_GHA_20010101_06h.nc')

ds

Copying gs://melvin_aims_bucket/evaporation/2001/ngcm_evaporation_2.8deg_6h_GHA_20010101_06h.nc...
/ [1 files][ 42.8 KiB/ 42.8 KiB]                                                
Operation completed over 1 objects/42.8 KiB.                                     


<ipython-input-57-7550eca587bc>:7: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset('/tmp/ngcm_evaporation_2.8deg_6h_GHA_20010101_06h.nc')


<xarray.Dataset> Size: 43kB
Dimensions:               (time: 37, surface: 1, longitude: 16, latitude: 18)
Coordinates:
  * time                  (time) datetime64[ns] 296B 2001-01-01 ... 2001-12-27
  * latitude              (latitude) float64 144B -18.14 -15.35 ... 26.51 29.3
  * longitude             (longitude) float64 128B 16.88 19.69 ... 56.25 59.06
  * surface               (surface) int32 4B 1
    prediction_timedelta  timedelta64[ns] 8B ...
    date                  datetime64[ns] 8B ...
Data variables:
    evaporation           (time, surface, longitude, latitude) float32 43kB ...

In [63]:
ds.latitude.values

array([-18.13897099, -15.34836476, -12.55775612,  -9.76714556,
        -6.97653355,  -4.18592053,  -1.39530691,   1.39530691,
         4.18592053,   6.97653355,   9.76714556,  12.55775612,
        15.34836476,  18.13897099,  20.92957425,  23.72017393,
        26.51076933,  29.30135962])

In [64]:
ds.longitude.values

array([16.875 , 19.6875, 22.5   , 25.3125, 28.125 , 30.9375, 33.75  ,
       36.5625, 39.375 , 42.1875, 45.    , 47.8125, 50.625 , 53.4375,
       56.25  , 59.0625])

In [66]:
ds.time.values

array(['2001-01-01T00:00:00.000000000', '2001-01-11T00:00:00.000000000',
       '2001-01-21T00:00:00.000000000', '2001-01-31T00:00:00.000000000',
       '2001-02-10T00:00:00.000000000', '2001-02-20T00:00:00.000000000',
       '2001-03-02T00:00:00.000000000', '2001-03-12T00:00:00.000000000',
       '2001-03-22T00:00:00.000000000', '2001-04-01T00:00:00.000000000',
       '2001-04-11T00:00:00.000000000', '2001-04-21T00:00:00.000000000',
       '2001-05-01T00:00:00.000000000', '2001-05-11T00:00:00.000000000',
       '2001-05-21T00:00:00.000000000', '2001-05-31T00:00:00.000000000',
       '2001-06-10T00:00:00.000000000', '2001-06-20T00:00:00.000000000',
       '2001-06-30T00:00:00.000000000', '2001-07-10T00:00:00.000000000',
       '2001-07-20T00:00:00.000000000', '2001-07-30T00:00:00.000000000',
       '2001-08-09T00:00:00.000000000', '2001-08-19T00:00:00.000000000',
       '2001-08-29T00:00:00.000000000', '2001-09-08T00:00:00.000000000',
       '2001-09-18T00:00:00.000000000', '2001-09-28

In [65]:
ds.date.values

numpy.datetime64('2001-01-01T06:00:00.000000000')

In [67]:
print(ds.coords)

Coordinates:
  * time                  (time) datetime64[ns] 296B 2001-01-01 ... 2001-12-27
  * latitude              (latitude) float64 144B -18.14 -15.35 ... 26.51 29.3
  * longitude             (longitude) float64 128B 16.88 19.69 ... 56.25 59.06
  * surface               (surface) int32 4B 1
    prediction_timedelta  timedelta64[ns] 8B ...
    date                  datetime64[ns] 8B 2001-01-01T06:00:00


In [70]:
print(ds.dims)

FrozenMappingWarningOnValuesAccess({'time': 37, 'surface': 1, 'longitude': 16, 'latitude': 18})


In [71]:
print(ds['evaporation'].attrs)

{}


In [74]:
print(lat)
print(lon)
print(time)
ds['evaporation'].sel(latitude=lat, longitude=lon, time=time).values

29.301359621762735
59.06250000000001
2001-12-27T00:00:00.000000000


array([-8.08953e-06], dtype=float32)

In [62]:
value

array([-8.08953e-06], dtype=float32)

In [56]:
# prompt: afficher les differentes valeurs qui y sont presenter dans ds je veux acceder a chacune des valeurs en fonction de la latitidet et autres

# Accéder aux valeurs en fonction de la latitude, longitude et temps :

for lat in ds.latitude.values:
  for lon in ds.longitude.values:
    for time in ds.time.values:
      value = ds['evaporation'].sel(latitude=lat, longitude=lon, time=time).values
      print(f"Latitude: {lat}, Longitude: {lon}, Time: {time}, Evaporation: {value}")


# Alternativement, vous pouvez utiliser l'indexation numérique :

for i in range(len(ds.latitude)):
  for j in range(len(ds.longitude)):
    for k in range(len(ds.time)):
      value = ds['evaporation'][k, i, j].values
      print(f"Latitude: {ds.latitude[i].values}, Longitude: {ds.longitude[j].values}, Time: {ds.time[k].values}, Evaporation: {value}")



# Pour afficher toutes les variables et leurs valeurs :

for var_name in ds.data_vars:
  print(f"Variable: {var_name}")
  print(ds[var_name].values)


# Pour afficher les attributs d'une variable spécifique (par exemple, 'evaporation') :

print()


# Pour afficher les coordonnées du dataset :



#Pour afficher les dimensions du dataset :


Streaming output truncated to the last 5000 lines.
Latitude: 20.929574254489513, Longitude: 39.375, Time: 2001-11-17T00:00:00.000000000, Evaporation: [-2.6454767e-05]
Latitude: 20.929574254489513, Longitude: 39.375, Time: 2001-11-27T00:00:00.000000000, Evaporation: [-3.2086402e-05]
Latitude: 20.929574254489513, Longitude: 39.375, Time: 2001-12-07T00:00:00.000000000, Evaporation: [-6.84605e-05]
Latitude: 20.929574254489513, Longitude: 39.375, Time: 2001-12-17T00:00:00.000000000, Evaporation: [-3.9368537e-05]
Latitude: 20.929574254489513, Longitude: 39.375, Time: 2001-12-27T00:00:00.000000000, Evaporation: [-2.9954965e-05]
Latitude: 20.929574254489513, Longitude: 42.18749999999999, Time: 2001-01-01T00:00:00.000000000, Evaporation: [-2.8925053e-06]
Latitude: 20.929574254489513, Longitude: 42.18749999999999, Time: 2001-01-11T00:00:00.000000000, Evaporation: [2.1102105e-07]
Latitude: 20.929574254489513, Longitude: 42.18749999999999, Time: 2001-01-21T00:00:00.000000000, Evaporation: [-8.0154

IndexError: range object index out of range